In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Shape and num of Pant Types
input_shape = (64, 64, 3)
num_classes = 3

# Create CNN
model = models.Sequential([
    # layer 1 / MaxPool
    layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),

    # layer 2 / Maxpool
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Add dense and dropout layers
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

2025-05-05 10:41:22.633921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)

In [3]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Directory
base_dir = os.path.expanduser("~/Google Drive/My Drive/ANN Project/Project Images")
train_dir = os.path.join(base_dir, "Training")    
test_dir  = os.path.join(base_dir, "Testing")

# 80/20 train/val split and preprocess
train_datagen = ImageDataGenerator(
    rescale=1./255,           
    rotation_range=10,        
    width_shift_range=0.1,    
    height_shift_range=0.1,  
    shear_range=0.1,          
    zoom_range=0.1,          
    horizontal_flip=True,     
    validation_split=0.2      
)

# Training
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64,64),
    batch_size=16,
    class_mode="categorical",
    classes=["PantA","PantB","PantC"],
    subset="training",
    shuffle=True
)

# Validating
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64,64),
    batch_size=16,
    class_mode="categorical",
    classes=["PantA","PantB","PantC"],
    subset="validation",
    shuffle=True
)

cwd = os.getcwd()
print("Current working directory:", cwd)
print("Writeable here?", os.access(cwd, os.W_OK))

models_dir = os.path.expanduser("~/models")
os.makedirs(models_dir, exist_ok=True)
print("Saving models into:", models_dir)

model_path = os.path.join(models_dir, "best_model.h5")

checkpoint = ModelCheckpoint(filepath = model_path, monitor="val_loss", save_best_only=True)
earlystop   = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    callbacks=[checkpoint, earlystop]
)

Found 1067 images belonging to 3 classes.
Found 265 images belonging to 3 classes.
Current working directory: /Users/jonathanewanchuk/Downloads
Writeable here? True
Saving models into: /Users/jonathanewanchuk/models


2025-05-05 10:41:33.783432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 0.8992 - accuracy: 0.5961

2025-05-05 10:41:39.137385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


67/67 [==============================] - 6s 76ms/step - loss: 0.8992 - accuracy: 0.5961 - val_loss: 0.8601 - val_accuracy: 0.6113
Epoch 2/15
67/67 [==============================] - 4s 58ms/step - loss: 0.7672 - accuracy: 0.6317 - val_loss: 0.9085 - val_accuracy: 0.7811
Epoch 3/15
67/67 [==============================] - 4s 61ms/step - loss: 0.7401 - accuracy: 0.6392 - val_loss: 0.8678 - val_accuracy: 0.6528
Epoch 4/15
67/67 [==============================] - 4s 60ms/step - loss: 0.7199 - accuracy: 0.6607 - val_loss: 0.8357 - val_accuracy: 0.7245
Epoch 5/15
67/67 [==============================] - 6s 95ms/step - loss: 0.7062 - accuracy: 0.6626 - val_loss: 0.6942 - val_accuracy: 0.7094
Epoch 6/15
67/67 [==============================] - 6s 87ms/step - loss: 0.6817 - accuracy: 0.6795 - val_loss: 0.8161 - val_accuracy: 0.7660
Epoch 7/15
67/67 [==============================] - 4s 67ms/step - loss: 0.6789 - accuracy: 0.6776 - val_loss: 0.6314 - val_accuracy: 0.7774
Epoch 8/15
67/67 [======

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Testing
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=(64,64),
    batch_size=16,
    class_mode="categorical",
    classes=["PantA","PantB","PantC"],
    shuffle=False
)

# Performance
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test loss: {test_loss:.3f}")
print(f"Test accuracy: {test_acc:.2%}")

# Predictions
y_prob = model.predict(test_gen)
y_pred = np.argmax(y_prob, axis=1)
y_true = test_gen.classes
class_names = list(test_gen.class_indices.keys())
num_classes = len(class_names)

# Confusion Matrix
cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=num_classes).numpy()
print("\nConfusion Matrix:")
print(cm)

# Per-class precision, recall, F1
print("\nClassification Report:")
print(f"{'Class':<8s}  {'Prec':>5s}  {'Rec':>5s}  {'F1':>5s}  {'Support':>7s}")
for i, name in enumerate(class_names):
    tp = cm[i,i]
    fp = cm[:,i].sum() - tp
    fn = cm[i,:].sum() - tp
    support = cm[i,:].sum()
    precision = tp / (tp + fp + 1e-8)
    recall    = tp / (tp + fn + 1e-8)
    f1        = 2 * (precision * recall) / (precision + recall + 1e-8)
    print(f"{name:<8s}  {precision:5.2f}  {recall:5.2f}  {f1:5.2f}  {support:7d}")


Found 148 images belonging to 3 classes.
 3/10 [========>.....................] - ETA: 0s - loss: 0.6982 - accuracy: 0.6875

2025-05-05 10:42:46.451021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 0s 24ms/step - loss: 0.6183 - accuracy: 0.7770
Test loss: 0.618
Test accuracy: 77.70%


2025-05-05 10:42:46.805257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 0s 20ms/step

Confusion Matrix:
[[12 11  4]
 [ 3 84  3]
 [ 1 11 19]]

Classification Report:
Class      Prec    Rec     F1  Support
PantA      0.75   0.44   0.56       27
PantB      0.79   0.93   0.86       90
PantC      0.73   0.61   0.67       31
